# 範例 : 計程車費率預測
https://www.kaggle.com/c/new-york-city-taxi-fare-prediction

# [作業目標]
- 試著模仿範例寫法, 使用程車費率預測競賽練習時間欄位處理

# [作業重點]
- 新增星期幾(day of week)與第幾周(week of year)這兩項特徵, 觀察有什麼影響 (In[4], Out[4], In[5], Out[5])
- 新增加上年週期與周周期特徵 , 觀察有什麼影響 (In[8], Out[8], In[9], Out[9]) 

In [1]:
# 做完特徵工程前的所有準備
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

data_path = 'data/'
df = pd.read_csv(data_path + 'taxi_data1.csv')

train_Y = df['fare_amount']
df = df.drop(['fare_amount'] , axis=1)
df.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2011-10-21 23:54:10 UTC,-73.990580,40.761071,-73.981128,40.758634,2
1,2015-02-03 10:42:03 UTC,-73.988403,40.723431,-73.989647,40.741695,1
2,2014-03-16 18:58:58 UTC,-74.015785,40.715110,-74.012029,40.707888,2
3,2009-06-13 16:10:54 UTC,-73.977322,40.787275,-73.958030,40.778838,3
4,2014-06-12 03:25:56 UTC,-73.989683,40.729717,-73.982490,40.761887,3


In [2]:
# 時間特徵分解方式:使用datetime
df['pickup_datetime'] = df['pickup_datetime'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S UTC'))
df['pickup_year'] = df['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%Y')).astype('int64')
df['pickup_month'] = df['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%m')).astype('int64')
df['pickup_day'] = df['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%d')).astype('int64')
df['pickup_hour'] = df['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%H')).astype('int64')
df['pickup_minute'] = df['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%M')).astype('int64')
df['pickup_second'] = df['pickup_datetime'].apply(lambda x: datetime.datetime.strftime(x, '%S')).astype('int64')
df.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,pickup_month,pickup_day,pickup_hour,pickup_minute,pickup_second
0,2011-10-21 23:54:10,-73.990580,40.761071,-73.981128,40.758634,2,2011,10,21,23,54,10
1,2015-02-03 10:42:03,-73.988403,40.723431,-73.989647,40.741695,1,2015,2,3,10,42,3
2,2014-03-16 18:58:58,-74.015785,40.715110,-74.012029,40.707888,2,2014,3,16,18,58,58
3,2009-06-13 16:10:54,-73.977322,40.787275,-73.958030,40.778838,3,2009,6,13,16,10,54
4,2014-06-12 03:25:56,-73.989683,40.729717,-73.982490,40.761887,3,2014,6,12,3,25,56


In [3]:
# 將結果使用線性迴歸 / 梯度提升樹分別看結果
df_temp = df.drop(['pickup_datetime'] , axis=1)
scaler = MinMaxScaler()
train_X = scaler.fit_transform(df_temp)
Linear = LinearRegression()
print(f'Linear Reg Score : {cross_val_score(Linear, train_X, train_Y, cv=5).mean()}')
GDBT = GradientBoostingRegressor()
print(f'Gradient Boosting Reg Score : {cross_val_score(GDBT, train_X, train_Y, cv=5).mean()}')

Linear Reg Score : 0.026876871475641616
Gradient Boosting Reg Score : 0.7105647001159359


# 作業1
* 對照範例，試著加入星期幾 (day of week) 與第幾周 (week of year) 這兩項特徵，  
看看結果會比原本只有時間特徵分解的結果更好或更差?

In [4]:
# 加上"星期幾 (day of week)"" 與"第幾周 (week of year)"" 特徵
df_temp = df.drop(['pickup_datetime'] , axis=1)

df_temp['day of week']=df['pickup_datetime'].apply(lambda x:x.weekday())
df_temp['week of year']=df['pickup_datetime'].apply(lambda x:x.strftime("%U"))

In [5]:
df_temp

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,pickup_month,pickup_day,pickup_hour,pickup_minute,pickup_second,day of week,week of year
0,-73.990580,40.761071,-73.981128,40.758634,2,2011,10,21,23,54,10,4,42
1,-73.988403,40.723431,-73.989647,40.741695,1,2015,2,3,10,42,3,1,05
2,-74.015785,40.715110,-74.012029,40.707888,2,2014,3,16,18,58,58,6,11
3,-73.977322,40.787275,-73.958030,40.778838,3,2009,6,13,16,10,54,5,23
4,-73.989683,40.729717,-73.982490,40.761887,3,2014,6,12,3,25,56,3,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,-73.994150,40.761397,-73.986840,40.759202,1,2011,12,23,19,7,0,4,51
4996,-73.986767,40.729937,-73.977022,40.726482,1,2009,8,26,18,19,41,2,34
4997,-73.984612,40.732297,-73.995004,40.750059,1,2010,10,25,6,5,21,0,43
4998,-73.958460,40.773415,-73.953445,40.782598,1,2010,2,16,16,57,0,1,07


In [6]:
# 將結果使用線性迴歸 / 梯度提升樹分別看結果

train_X = scaler.fit_transform(df_temp)
print(f'Linear Reg Score : {cross_val_score(Linear, train_X, train_Y, cv=5).mean()}')
print(f'Gradient Boosting Reg Score : {cross_val_score(GDBT, train_X, train_Y, cv=5).mean()}')

Linear Reg Score : 0.026467123376530944
Gradient Boosting Reg Score : 0.7152381681341823


In [7]:
# 加上"日週期"特徵 (參考講義"週期循環特徵")
import math
df_temp['day_cycle'] = df_temp['pickup_hour']/12 + df_temp['pickup_minute']/720 + df_temp['pickup_second']/43200
df_temp['day_cycle'] = df_temp['day_cycle'].map(lambda x:math.sin(x*math.pi))
df_temp.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,pickup_month,pickup_day,pickup_hour,pickup_minute,pickup_second,day of week,week of year,day_cycle
0,-73.990580,40.761071,-73.981128,40.758634,2,2011,10,21,23,54,10,4,42,-0.025450
1,-73.988403,40.723431,-73.989647,40.741695,1,2015,2,3,10,42,3,1,05,0.333601
2,-74.015785,40.715110,-74.012029,40.707888,2,2014,3,16,18,58,58,6,11,-0.967083
3,-73.977322,40.787275,-73.958030,40.778838,3,2009,6,13,16,10,54,5,23,-0.888817
4,-73.989683,40.729717,-73.982490,40.761887,3,2014,6,12,3,25,56,3,23,0.782427


In [8]:
# 將結果使用線性迴歸 / 梯度提升樹分別看結果

train_X = scaler.fit_transform(df_temp)
print(f'Linear Reg Score : {cross_val_score(Linear, train_X, train_Y, cv=5).mean()}')
print(f'Gradient Boosting Reg Score : {cross_val_score(GDBT, train_X, train_Y, cv=5).mean()}')

Linear Reg Score : 0.026048223699883178
Gradient Boosting Reg Score : 0.7102873846672656


# 作業2
* 對照範例的日週期效果，試著參考投影片完成年週期與周週期的特徵 (也可以用你自己想到的方式)，  
看看結果會比範例中的結果更好或更差?

In [9]:
#年週期
df_temp['year_cycle'] = (df_temp['pickup_month']/6) + (df_temp['pickup_day']/180) 
df_temp['year_cycle'] = df_temp['year_cycle'].map(lambda x:math.cos(x*math.pi))

#周週期
df_temp['week_cycle'] = df_temp['day of week']/(3.5) + df_temp['pickup_hour']/84
df_temp['week_cycle'] = df_temp['week_cycle'].map(lambda x:math.sin(x*math.pi))

In [10]:
df_temp

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,pickup_month,pickup_day,pickup_hour,pickup_minute,pickup_second,day of week,week of year,day_cycle,year_cycle,week_cycle
0,-73.990580,40.761071,-73.981128,40.758634,2,2011,10,21,23,54,10,4,42,-0.025450,0.777146,-0.965926
1,-73.988403,40.723431,-73.989647,40.741695,1,2015,2,3,10,42,3,1,05,0.333601,0.453990,0.955573
2,-74.015785,40.715110,-74.012029,40.707888,2,2014,3,16,18,58,58,6,11,-0.967083,-0.275637,-0.222521
3,-73.977322,40.787275,-73.958030,40.778838,3,2009,6,13,16,10,54,5,23,-0.888817,-0.974370,-0.930874
4,-73.989683,40.729717,-73.982490,40.761887,3,2014,6,12,3,25,56,3,23,0.782427,-0.978148,0.330279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,-73.994150,40.761397,-73.986840,40.759202,1,2011,12,23,19,7,0,4,51,-0.957571,0.920505,-0.916562
4996,-73.986767,40.729937,-73.977022,40.726482,1,2009,8,26,18,19,41,2,34,-0.996314,-0.069756,0.623490
4997,-73.984612,40.732297,-73.995004,40.750059,1,2010,10,25,6,5,21,0,43,0.999728,0.819152,0.222521
4998,-73.958460,40.773415,-73.953445,40.782598,1,2010,2,16,16,57,0,1,07,-0.962455,0.241922,0.997204


In [11]:
# 將結果使用線性迴歸 / 梯度提升樹分別看結果

train_X = scaler.fit_transform(df_temp)
print(f'Linear Reg Score : {cross_val_score(Linear, train_X, train_Y, cv=5).mean()}')
print(f'Gradient Boosting Reg Score : {cross_val_score(GDBT, train_X, train_Y, cv=5).mean()}')

Linear Reg Score : 0.025984508659212112
Gradient Boosting Reg Score : 0.7118232025007301


In [12]:
## 只做day of week &week of year 分數較高 再加上週期特徵反而變差